# Knihovna threading
Threading je modul, který umožňuje v Pythonu vytvářet vlákna. Tedy jednotlivé výpočetní čísti, které v rámci jednoho procesu (zde kernelu/interpretu) sdílejí data.

Jak už bylo zmíněno, v Pythonu existuje GIL (Global interpret lock) který umožňuje v jednu chvíli pouze jednomu vláknu vykonávat python kód. Toto je sice nevýhoda z pohledu výkonu, ale na úrovni pythonu za nás řeší veškeré neduhy, které by mohly nastat při sdílení dat mezi vlákny (race conditions, deadlocks, ...).

Threading nabízí mnoho dalších funkcionalit, viz [dokumentace](https://docs.python.org/3/library/threading.html):
- `threading.Condition` - daší způsob jak synchronizovat vlákna
- `threading.Semaphore` - synchronizace vláken pomocí semaforů
- `threading.Timer` - časovač, který spustí funkci po určité době
- `threading.Event` - synchronizace vláken pomocí událostí


## Vytváření vláken
Pro vytváření vláken je potřeba vytvořit instanci třídy `Thread`. Tato třída má několik parametrů, ale nejdůležitější je parametr `target`, který určuje funkci, která se má vykonávat v novém vlákně.

Další parametry konstruktoru jsou:
- `args`/`kwargs` - tuple/slovník argumentů, které se mají předat funkci
- `name` - jméno vlákna

Vlákna mají několik metod, které jsou pro nás důležité:
- `start()` - spustí vlákno
- `join()` - čeká na ukončení vlákna
- `is_alive()` - vrací `True` pokud je vlákno aktivní


In [ ]:
import threading

def some_function():
    # kód, který se má provést ve vlákně
    print("Hello from thread!")

thread = threading.Thread(target=some_function)
thread.start()  # spustí vlákno
thread.join()  # čeká na dokončení vlákna


Vyzkoušíme si předat vláknu argumenty:

In [ ]:
import threading
import time

def vypocet(results, id):
    a = 0
    for i in range(100000000):
        a += i
    results[id] = a

zacatek = time.time()

results = [0]

vlakno1 = threading.Thread(target=vypocet, args=(results, 0))

vlakno1.start()

vlakno1.join()

konec = time.time()
print("Doba trvání: ", konec - zacatek)


In [ ]:
import threading
import time

def vypocet(results, id):
    a = 0
    for i in range(100000000):
        a += i
    results[id] = a

zacatek = time.time()

results = [0, 0]

vlakno1 = threading.Thread(target=vypocet, args=(results, 0))
vlakno2 = threading.Thread(target=vypocet, kwargs={"results": results, "id": 1})

vlakno1.start()
vlakno2.start()

vlakno1.join()
vlakno2.join()

konec = time.time()
print("Doba trvání: ", konec - zacatek)


Ukázali jsme si, že vlákna nejsou schopny spouštět python kód zároveň a tedy předchozí ukázka s dvěmi vlákny trvá 2x déle.

#### Vlákna, GIL, a spouštění kódu mimo python
Ukážeme si podobnou ukázku jako minule, ale nyní spustíme ve vláknech funkci zkompilovanou pomocí Numby, nastavenou tak ať releasne (propustí) GIL.

In [ ]:
from numba import jit

@jit(nogil=True)
def core(): # nějaká funkce, dělající netriviální výpočty, ať je co měřit
    a = 0
    n = int(1e5)
    for i in range(n):
        for j in range(n):
            a += 2*i*j
            a -= i*i
            a -= j*j
    return a

In [ ]:
%time res = core()

In [ ]:
res

In [ ]:
def vypocet(results, idx):
    a = core()
    results[idx] = a

zacatek = time.time()

results = [0, 0]
vlakno1 = threading.Thread(target=vypocet, args=(results, 0))
vlakno2 = threading.Thread(target=vypocet, args=(results, 1))

vlakno1.start()
vlakno2.start()

vlakno1.join()
vlakno2.join()

konec = time.time()
print("Doba trvání: ", konec - zacatek)
results



## Vlákna a sdílení dat

In [ ]:
import threading
import time
import random

data = []  # sdílená datová struktura

def append_to_data(item):
    time.sleep(random.randint(0, 10)/1000)
    data.append(item)

items = [i for i in range(20)]

threads = []
for item in items:
    thread = threading.Thread(target=append_to_data, args=(item,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

print(f"Sdílená data: {data}")


## Prioritní exekuce vlákna
Vlákno si může vyžádat GIL na nějakou dobu pro sebe (`lock`). Pak má zajištěno, že blok kódu, který je v `lock`u, bude vykonán bez přerušení jiným vláknem. Toto je užitečné například při sdílení dat mezi vlákny.

In [ ]:
import threading

lock = threading.Lock()  # vytvoření zámku
shared_resource = 0

def increment_shared_resource():
    global shared_resource
    with lock:  # získání zámku
        shared_resource += 1

threads = []
for _ in range(10):
    thread = threading.Thread(target=increment_shared_resource)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

print(f"Hodnota sdíleného zdroje: {shared_resource}")


In [ ]:
import threading
import time

def vypis(thread_id):
    for i in range(5):
        time.sleep(0.001)
        print(f"Vlákno {thread_id} vypisuje {i}")

threads = []
for idx in range(5):
    thread = threading.Thread(target=vypis, args=(idx,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

In [ ]:
import threading
import time
lock = threading.Lock()  # vytvoření zámku

def vypis(thread_id):
    with lock:  # získání zámku
        for i in range(5):
            time.sleep(0.001)
            print(f"Vlákno {thread_id} vypisuje {i}")

threads = []
for idx in range(5):
    thread = threading.Thread(target=vypis, args=(idx,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

## Synchronizace mezi vlákny = Barrier
K synchronizaci mezi vlákny je k dispozici třída `Barrier`. `Barrier` je synchronizační mechanismus, který umožňuje skupině vláken čekat na sebe navzájem, dokud všechna vlákna nedosáhnou určitého bodu (bariéry). Jakmile dosáhnou všechna vlákna bariéry, mohou pokračovat ve své práci.

Použití Barrier je jednoduché. Nejprve vytvořte instanci `Barrier` s počtem vláken, která mají čekat na bariéře. Poté se tato instance pošle do vlákna (funkce kterou vykonává) jako parametr. Bariéra se vyvolá použitím metody `wait()`.

In [ ]:
import threading
import time

def worker(barrier, worker_id):
    print(f"Vlákno {worker_id} začíná.")
    time.sleep(worker_id)  # simulace různých dob vykonávání vláken
    print(f"Vlákno {worker_id} dosáhlo bariéry č. 1.")
    barrier.wait()
    print(f"Vlákno {worker_id} opouští bariéru č. 1.")

    time.sleep(worker_id)  # simulace různých dob vykonávání vláken
    print(f"Vlákno {worker_id} dosáhlo bariéry č. 2.")
    barrier.wait()
    print(f"Vlákno {worker_id} opouští bariéru č. 2.")

number_of_threads = 5
barrier = threading.Barrier(number_of_threads)

threads = []
for i in range(number_of_threads):
    thread = threading.Thread(target=worker, args=(barrier, i))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()
